In [ ]:
%%html
<style>
.output_subarea.output_text.output_stream.output_stdout > pre {
    width:max-content;
}
.p-Widget.jp-RenderedText.jp-OutputArea-output > pre {
   width:max-content;
}
</style>

In [ ]:
import sys
import os
import operator

from pyspark.sql import Row
from pyspark.sql.functions import udf, when, rand, struct, col, lit, current_date, current_timestamp, monotonically_increasing_id
from pyspark.sql.types import StringType, IntegerType, StructField, StructType, DoubleType, ArrayType, FloatType

In [ ]:
import visualize
from com.ophelia.OpheliaVendata import OpheliaVendata

In [ ]:
module_path = os.path.abspath(os.path.join('..'))

In [ ]:
ophelia = OpheliaVendata("Risk Classification Demo")
sc = ophelia.ophelia_spark.build_spark_context()
sc.addFile(module_path+"/enquire/", recursive = True)

In [ ]:
spark = ophelia.ophelia_session
path = "data/master/ophelia/data/OpheliaData/risk_classification/"
customer_data = spark.read.parquet(path)
customer_data.printSchema()
customer_data.show(5, False)

In [ ]:
#develop/ophelia/ophelia/data/master/ophelia/data/OpheliaData/risk_classification


#customer_data = spark.read.csv('data/raw/csv/bank.csv', header='true')
#customer_data.show()

#writ = customer_data.select("age", "job", "marital", "education")\
#                    .withColumn("gender", when(rand() > 0.5, "female").otherwise("male"))\
#                    .withColumn("child", when(rand() > 0.5, float(1.0)).otherwise(float(0.0)).cast('float'))\
#                    .withColumn("saving", when(rand() > 0.5, float(1.0)).otherwise(float(0.0)).cast('float'))\
#                    .withColumn("insight", when(rand() > 0.5, float(1.0)).otherwise(float(0.0)).cast('float'))\
#                    .withColumn("backup", when(rand() > 0.5, float(1.0)).otherwise(float(0.0)).cast('float'))
#writ.show()
#writ.printSchema()

In [ ]:
#writ.coalesce(1).write.mode("overwrite").parquet("data/master/ophelia/data/OpheliaData/risk_classification/")

In [ ]:
column_selection = [
    col('age').cast('int').alias('age'),
    col('job').cast('string').alias('job'),
    col('marital').cast('string').alias('marital'),
    col('education').cast('string').alias('education'),
    col('gender').cast('string').alias('gender'),
    col('child').cast('float').alias('child'),
    col('saving').cast('float').alias('saving'),
    col('insight').cast('float').alias('insight'),
    col('backup').cast('float').alias('backup')
]

struct_assembler = customer_data.select(*column_selection, struct(column_selection).alias("struct_assembler"))
struct_assembler.show(5, False)
struct_assembler.printSchema()

In [ ]:
def build_tree_classifier():
    schema_tree = ArrayType(StructType([
        StructField("vote", FloatType(), False),
        StructField("weight", FloatType(), False),
        StructField("risk_label", StringType(), False)
    ]))
    return udf(ophelia.ophelia_classifier.tree_generator, schema_tree)

def build_risk_class():
    return udf(ophelia.ophelia_classifier.run_classification_risk, StringType())

In [ ]:
tree_model_udf = build_tree_classifier()
risk_class_udf = build_risk_class()

In [ ]:
#schema_tree = ArrayType(StructType([
#    StructField("vote", FloatType(), False),
#    StructField("weight", FloatType(), False),
#    StructField("risk_label", StringType(), False)
#]))
#
#tree_udf = udf(ophelia.ophelia_classifier.tree_generator, schema_tree)
#classification_udf = udf(ophelia.ophelia_classifier.run_classification_risk, StringType())

In [ ]:
tree_model_df = struct_assembler.select("*", tree_model_udf(col("struct_assembler")).alias("tree")).cache()
tree_model_df.show(5, False)
tree_model_df.printSchema()

In [ ]:
risk_class_df = tree_model_df.select(risk_class_udf(col("tree")).alias("risk_label"),
                                     *column_selection, current_date().alias("information_date"),
                                     current_timestamp().alias("model_date"))

risk_class_df.groupBy("risk_label").count().show(10, False)
risk_class_df.show(10, False)
risk_class_df.printSchema()

In [ ]:
columns_risk = risk_class_df.columns
when_conditions = (
    when(col("risk_label") == "A", lit(1.0)).otherwise(
        when(col("risk_label") == "MA", lit(2.0)).otherwise(
            when(col("risk_label") == "M", lit(3.0)).otherwise(
                when(col("risk_label") == "MC", lit(4.0)).otherwise(
                    when(col("risk_label") == "C", lit(5.0))))))
)
mapped_risk_label = risk_class_df.select(when_conditions.alias("risk_label_id"), *columns_risk,
                                         (monotonically_increasing_id + 100000000).alias("customer_id"))
mapped_risk_label.show(5, False)

In [ ]:
riskClassPath = ophelia.ophelia_write.write_parquet(
    df=mapped_risk_label,
    output_type="model",
    project="RiskClassifier",
    part="information_date"
)